In [2]:
import pandas as pd
import torch

df_fake = pd.read_csv(r"C:\Users\16784\Downloads\archive (3)\Fake.csv")
df_true = pd.read_csv(r"C:\Users\16784\Downloads\archive (3)\True.csv")

from transformers import LongformerTokenizer

# Load RoBERTa tokenizer
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

# Concatenate title and text columns
df_fake['input_text'] = df_fake['title'] + " " + df_fake['text']
df_true['input_text'] = df_true['title'] + " " + df_true['text']

import re

def clean_text(text):
    # Remove HTML tags
    cleaned_text = re.sub('<.*?>', '', text)
    # Remove special characters, punctuation, and non-alphanumeric characters
    cleaned_text = re.sub(r'[^\w\s]', '', cleaned_text)
    # Remove extra whitespaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    # To lower case
    cleaned_text = cleaned_text.lower()
    return cleaned_text

df_fake['input_text'] = df_fake['input_text'].apply(clean_text)
df_true['input_text'] = df_true['input_text'].apply(clean_text)

C:\Users\16784\PycharmProjects\fakeNews\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

C:\Users\16784\PycharmProjects\fakeNews\.venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\16784\.cache\huggingface\hub\models--allenai--longformer-base-4096. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [3]:
def tokenize_and_encode(text):
    # Tokenize and encode text with padding, truncation, and set the return type to PyTorch tensor
    tokens = tokenizer(text, padding="max_length", truncation=True, max_length=4096, return_tensors="pt")

    # Initialize a global attention mask and set global attention on the first token
    tokens['global_attention_mask'] = torch.zeros_like(tokens['attention_mask'])
    tokens['global_attention_mask'][:, 0] = 1  # Typically, global attention on the first token

    return {
        'input_ids': tokens['input_ids'][0],  # Extract the tensor from the batch
        'attention_mask': tokens['attention_mask'][0],
        'global_attention_mask': tokens['global_attention_mask'][0]
    }

df_fake['encoded_text'] = df_fake['input_text'].apply(tokenize_and_encode)
df_true['encoded_text'] = df_true['input_text'].apply(tokenize_and_encode)

# Create labels
df_fake['label'] = 0  # Fake news
df_true['label'] = 1  # True news

df = pd.concat([df_fake, df_true], axis = 0, ignore_index = True)



df_shuffled = df.sample(frac=1).reset_index(drop=True) # shuffle

In [4]:
def tensor_to_list(row):
    return {
        'input_ids': row['input_ids'].tolist(),
        'attention_mask': row['attention_mask'].tolist(),
        'global_attention_mask': row['global_attention_mask'].tolist()
    }

# Apply the function to the dataframe
df_shuffled['encoded_text'] = df_shuffled['encoded_text'].apply(tensor_to_list)


In [5]:
df_shuffled = df_shuffled[['encoded_text', 'label']]

In [6]:
# the dataset is too big to fine tune so only sample 5% of them for testing purpose
df_shuffled = df_shuffled.sample(frac=1/20, random_state=42) 

In [7]:
# Define split ratios
train_size = int(0.8 * len(df_shuffled))
valid_size = int(0.1 * len(df_shuffled))

# Split the data
train_df = df_shuffled[:train_size]
valid_df = df_shuffled[train_size:train_size + valid_size]
test_df = df_shuffled[train_size + valid_size:]

In [8]:
train_df

,encoded_text,label
22216,"{'input_ids': [0, 20125, 12297, 727, 330, 31, ...",0
27917,"{'input_ids': [0, 4892, 5367, 4385, 5299, 7661...",0
25007,"{'input_ids': [0, 309, 7, 5, 92, 1104, 790, 83...",0
1377,"{'input_ids': [0, 40018, 61, 1939, 1984, 1240,...",0
32476,"{'input_ids': [0, 16111, 106, 160, 50, 582, 13...",0
...,...,...
29200,"{'input_ids': [0, 611, 1243, 19288, 1926, 449,...",1
37723,"{'input_ids': [0, 4897, 1494, 16857, 4267, 7, ...",1
39900,"{'input_ids': [0, 3916, 3141, 6344, 1263, 7, 1...",0
8040,"{'input_ids': [0, 38060, 26066, 1637, 397, 579...",1


In [9]:
def df_to_jsonlines(df, filename):
    """ Convert DataFrame to JSONLines file format """
    df.to_json(filename, orient='records', lines=True)

# Convert datasets to JSONLines
df_to_jsonlines(train_df, 'train.jsonl')
df_to_jsonlines(valid_df, 'valid.jsonl')
df_to_jsonlines(test_df, 'test.jsonl')

In [14]:
import boto3

# Function to upload files to S3
def upload_file_to_s3(file_name, bucket, object_name=None):
    if object_name is None:
        object_name = file_name
    s3_client = boto3.client('s3')
    response = s3_client.upload_file(file_name, bucket, object_name)
    return response

# Upload the training data
upload_file_to_s3('train.jsonl', 'fakenewspj', 'input/train.jsonl')
upload_file_to_s3('valid.jsonl', 'fakenewspj', 'input/valid.jsonl')
upload_file_to_s3('test.jsonl', 'fakenewspj', 'input/test.jsonl')

NoCredentialsError: Unable to locate credentials

In [2]:
%pip uninstall torch torchvision torchaudio


2.3.0+cpu
None
